# COBRApy Tutorial
The purpose of this workflow is to describe the basic functions and methods of COBRApy by creating a simple model containing 1 reaction.

COBRApy is a Python module that can be imported similar to any other python module

In [1]:
import cobra

Use tab complete to show basic classes that can be accessed through the module. By convention classes are capitalized. 

In [2]:
cobra

<module 'cobra' from '/home/sbrg-cjlloyd/Envs/qbio18/lib/python3.6/site-packages/cobra/__init__.py'>

These classes include:
1. `cobra.Reaction`
2. `cobra.Metabolite`
3. `cobra.Model`
4. `cobra.Gene`

-----
## A) Create an empty model

#### 1) Instantiate a new model by calling Model the model class using "`test_model`" as the model id. 

You can view the documentation using the following:

In [3]:
print(cobra.Model.__doc__)

Class representation for a cobra model

    Parameters
    ----------
    id_or_model : Model, string
        Either an existing Model object in which case a new model object is
        instantiated with the same properties as the original model,
        or a the identifier to associate with the model as a string.
    name : string
        Human readable name for the model

    Attributes
    ----------
    reactions : DictList
        A DictList where the key is the reaction identifier and the value a
        Reaction
    metabolites : DictList
        A DictList where the key is the metabolite identifier and the value a
        Metabolite
    genes : DictList
        A DictList where the key is the gene identifier and the value a
        Gene
    solution : Solution
        The last obtained solution from optimizing the model.
    


or

In [4]:
cobra.Model?

In [5]:
model = cobra.Model('test_model')

#### 2) See the contents of the model by excuting the model instance

In [6]:
model

Name,test_model
Memory address,0x07f680ca44278
Number of metabolites,0
Number of reactions,0
Objective expression,0
Compartments,


-----
## B) Create and add metabolites to model

#### 1) Like above, instantiate a new Metabolite object with ID '`f6p_c`'

 - By convention `_c` refers to the metabolites cellular compartment, in this case the cytosol 

In [7]:
print(cobra.Metabolite.__doc__)

Metabolite is a class for holding information regarding
    a metabolite in a cobra.Reaction object.

    Parameters
    ----------
    id : str
        the identifier to associate with the metabolite
    formula : str
        Chemical formula (e.g. H2O)
    name : str
        A human readable name.
    charge : float
       The charge number of the metabolite
    compartment: str or None
       Compartment of the metabolite.
    


In [8]:
metabolite_1 = cobra.Metabolite('f6p_c')

In [9]:
metabolite_1

Metabolite identifier,f6p_c
Name,
Memory address,0x07f680ca448d0
Formula,None
Compartment,None
In 0 reaction(s),


#### 2) Fill in the missing attributes with the following information

| Attribute       | Value       |    
| :-------------: |:-------------:|
| name | D-Fructose 6-phosphate     |
| compartment | Cytosol     |
| formula     | C6H11O9P |
| charge      | -2      | 

In [10]:
metabolite_1.name = 'D-Fructose 6-phosphate'
metabolite_1.compartment = 'Cytosol'
metabolite_1.formula = 'C6H11O9P'
metabolite_1.charge = -2

#### 3) See the contents of the metabolite by excuting the metabolite instance

In [11]:
metabolite_1

Metabolite identifier,f6p_c
Name,D-Fructose 6-phosphate
Memory address,0x07f680ca448d0
Formula,C6H11O9P
Compartment,Cytosol
In 0 reaction(s),


#### 4) New properties can be calculated from attributes. For example access the metabolites `formula_weight` property
 - Computed based on metabolite formula

In [12]:
metabolite_1.formula_weight

258.119901

#### 5) Create new metabolite with following attributes. Assign attributes when instantiating the metabolite 

| Attribute       | Value       |    
| :-------------: |:-------------:|
| id | g6p_c     |
| mame | D-Glusose 6-phosphate     |
| compartment | Cytosol     |
| formula     | C6H11O9P |
| charge      | -2      | 

In [13]:
metabolite_2 = cobra.Metabolite(id='g6p_c', name='D-Glucose 6-phosphate', compartment='Cytosol', 
                                formula='C6H11O9P', charge=-2)

In [14]:
metabolite_2

Metabolite identifier,g6p_c
Name,D-Glucose 6-phosphate
Memory address,0x07f680ca44eb8
Formula,C6H11O9P
Compartment,Cytosol
In 0 reaction(s),


#### 6) Add both metabolites to model container

As with classes, we can observe the properties of functions like above

In [15]:
model.add_metabolites?

In [16]:
model.add_metabolites([metabolite_1, metabolite_2])

#### 7) See contents of model after adding metabolites

In [17]:
model

Name,test_model
Memory address,0x07f680ca44278
Number of metabolites,2
Number of reactions,0
Objective expression,0
Compartments,Cytosol


-----
## C) Create and add reaction to model

#### 1) Like the model and metabolites, instantiate a reaction with id '`PGI`' and name '`Glucose-6-phosphate isomerase`'

In [18]:
reaction = cobra.Reaction(id='PGI', name='Glucose-6-phosphate isomerase')

In [19]:
reaction

Reaction identifier,PGI
Name,Glucose-6-phosphate isomerase
Memory address,0x07f680cabc518
Stoichiometry,--> -->
GPR,
Lower bound,0.0
Upper bound,1000.0


#### 2) Add the above metabolites to the reaction as a dictionary

In [20]:
stoichiometry = {'g6p_c': -1, 'f6p_c': 1}

In [21]:
reaction.add_metabolites(stoichiometry)

ValueError: Reaction 'PGI' does not belong to a model. Either add the reaction to a model or use Metabolite objects instead of strings as keys.

**Note:** This produces and error because the reaction has not been added to the model yet. This means the reaction cannot access the metabolite information that we added above. So we must use either of the two approaches outlined in the error message.

 - **Option 1**: Use metabolite objects as keys

In [22]:
metabolite_1 = model.metabolites.get_by_id('g6p_c')
metabolite_2 = model.metabolites.f6p_c

reaction.add_metabolites({metabolite_1: -1, metabolite_2: 1})

 - **Option 2**: Use add reaction to model first

In [23]:
model.add_reaction(reaction)

In [24]:
# combine = false to not add coefficients 
reaction.add_metabolites({'g6p_c': -1, 'f6p_c': 1}, combine=False)

#### 3) View contents of reaction

In [25]:
reaction

Reaction identifier,PGI
Name,Glucose-6-phosphate isomerase
Memory address,0x07f680cabc518
Stoichiometry,g6p_c --> f6p_c D-Glucose 6-phosphate --> D-Fructose 6-phosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


#### 4) View contents of model

In [26]:
model

Name,test_model
Memory address,0x07f680ca44278
Number of metabolites,2
Number of reactions,1
Objective expression,0
Compartments,Cytosol


In [27]:
print(model.metabolites)

[<Metabolite f6p_c at 0x7f680ca448d0>, <Metabolite g6p_c at 0x7f680ca44eb8>]


In [28]:
print(model.reactions)

[<Reaction PGI at 0x7f680cabc518>]


#### 5) Make the reaction reversible

The reaction has two attributes **`lower_bound`** and **`upper_bound`** which place limits on how much flux the reaction can carry in a simulation. By default, the reaction is set to be irreversible in the forward direction (1000 (w/ units of $\frac{mmol}{grams\_dry\_weight \cdot hr}$ ) is used to effectively represent unbounded possible forward flux).

 - To make the reaction reversible, set the **`lower_bound`** of the reaction to -1000

In [29]:
reaction.lower_bound = -1000.

#### 6) View contents of the reaction noting the arrow change

In [30]:
reaction

Reaction identifier,PGI
Name,Glucose-6-phosphate isomerase
Memory address,0x07f680cabc518
Stoichiometry,g6p_c <=> f6p_c D-Glucose 6-phosphate <=> D-Fructose 6-phosphate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


-----
## D) Associate genes to reaction

Genes are linked to reactions via a **`gene_reaction_rule`** or **`GPR`**, above. There are boolean "OR" "AND" relationships that determine whether a reaction is knocked out based on the presence of genes in the model.

#### 1) First set the GPR as an OR relationship with two genes (b1111 and b2222)

In [56]:
reaction.gene_reaction_rule = 'b1111 or b2222'

#### 2) View reaction to observe addition of GPR

In [52]:
reaction

Reaction identifier,PGI
Name,Glucose-6-phosphate isomerase
Memory address,0x07f680cabc518
Stoichiometry,g6p_c <=> f6p_c D-Glucose 6-phosphate <=> D-Fructose 6-phosphate
GPR,b1111 or b2222
Lower bound,-1000
Upper bound,10000


#### 3) Access gene through the model
 - Gene is added to model automatically through the reaction

In [55]:
model.genes.b1111

Gene identifier,b1111
Name,
Memory address,0x07f680ca4c5c0
Functional,True
In 1 reaction(s),PGI


#### 4) Simulate gene deletion by knocking out b1111 using the `knock_out()` gene method

In [57]:
model.genes.b1111.knock_out()

#### 5) View reaction to determine if knocked out

In [60]:
reaction

Reaction identifier,PGI
Name,Glucose-6-phosphate isomerase
Memory address,0x07f680cabc518
Stoichiometry,g6p_c <=> f6p_c D-Glucose 6-phosphate <=> D-Fructose 6-phosphate
GPR,b1111 or b2222
Lower bound,-1000
Upper bound,10000


In [61]:
False or True

True

#### 6) Change GPR to and relationship with same two genes

In [62]:
reaction.gene_reaction_rule = 'b1111 and b2222'

#### 7) Knock out b1111 again

In [65]:
model.genes.b1111.knock_out()

#### 8) View reaction again to determine if knocked out

In [66]:
reaction

Reaction identifier,PGI
Name,Glucose-6-phosphate isomerase
Memory address,0x07f680cabc518
Stoichiometry,g6p_c --> f6p_c D-Glucose 6-phosphate --> D-Fructose 6-phosphate
GPR,b1111 and b2222
Lower bound,0
Upper bound,0


In [67]:
False and True

False